# Computing relative hydration free energy between benzene and phenol

In [1]:
import os
import pickle
from itertools import product
import time
import openmm as mm
import openmm.app as app
import openmm.unit as unit
from openmm import XmlSerializer
import numpy as np
import mdtraj
import xml.etree.ElementTree as ET
from FastMBAR import FastMBAR
from openatom import (
    compute_mcs_VF2,
    make_graph,
    align_coordinates,
    make_alchemical_system,
    make_psf_from_topology, 
)

Most molecular dynamics software packages including OpenMM provides tools to setup physical systems
for simulations. However, in alchemical free energy calculations, physical systems are modified 
using alchemical transformations which yields systems that are not physical such as systems 
corresponding to the intermediate states of the transformation. The package `atom` provides tools 
to setup alchemical systems for simulations.

In this notebook, we will use `atom` to setup alchemical systems for computing the relative hydration
free energy between benzene and phenol. First, we will tools provided in `openmm` to make 
three systems: benzene in vacuum, phenol in vacuum, a water box. 

In [2]:
## Make an openmm system for the environment based on the prmtop file
## The environment is a solvent box
envi_prmtop = app.AmberPrmtopFile("./structures/solvent.prmtop")
envi_system = envi_prmtop.createSystem(
    nonbondedMethod=app.PME,
    nonbondedCutoff=1.0 * unit.nanometer,
    constraints=app.HBonds,
    switchDistance=0.9 * unit.nanometer,
)

## Topology and coordinates of the environment
envi_top = envi_prmtop.topology
envi_coor = app.AmberInpcrdFile("./structures/solvent.inpcrd").getPositions()
envi_coor = np.array(envi_coor.value_in_unit(unit.nanometer))


## Make an openmm system for benzene
## Note that we use the same periodic box vectors as the environment
liga_prmtop = app.AmberPrmtopFile(
    "./structures/BNZ.prmtop", envi_top.getPeriodicBoxVectors()
)
liga_system = liga_prmtop.createSystem(
    nonbondedMethod=app.PME,
    nonbondedCutoff=1.0 * unit.nanometer,
    constraints=app.HBonds,
    switchDistance=0.9 * unit.nanometer,
)

## Topology and coordinates of benzene
liga_top = liga_prmtop.topology
liga_coor = app.AmberInpcrdFile("./structures/BNZ.inpcrd").getPositions()
liga_coor = np.array(liga_coor.value_in_unit(unit.nanometer))


## Make an openmm system for phenol
## Make sure that we use the same periodic box vectors as the environment
ligb_prmtop = app.AmberPrmtopFile(
    "./structures/IPH.prmtop", envi_top.getPeriodicBoxVectors()
)
ligb_system = ligb_prmtop.createSystem(
    nonbondedMethod=app.PME,
    nonbondedCutoff=1.0 * unit.nanometer,
    constraints=app.HBonds,
    switchDistance=0.9 * unit.nanometer,
)

## Topology and coordinates of phenol
ligb_top = ligb_prmtop.topology
ligb_coor = app.AmberInpcrdFile("./structures/IPH.inpcrd").getPositions()
ligb_coor = np.array(ligb_coor.value_in_unit(unit.nanometer))

Using the three systems of benzene in vacuum, phenol in vacuum, and a water box, we will setup
alchemical systems for benzene and phenol in water.

In [3]:
## Serialize the systems to xml.
## `atom` creates alchemical systems by manipulating the xml representation of the systems
envi_xml = XmlSerializer.serializeSystem(envi_system)
liga_xml = XmlSerializer.serializeSystem(liga_system)
ligb_xml = XmlSerializer.serializeSystem(ligb_system)


envi = ET.fromstring(envi_xml)
liga = ET.fromstring(liga_xml)
ligb = ET.fromstring(ligb_xml)
ligs = [liga, ligb]

## compute the maximum common substructure between benzene and phenol using VF2 algorithm
mcs = compute_mcs_VF2(liga_top, ligb_top, timeout=30)

liga_common_atoms = list(mcs.keys())
ligb_common_atoms = [mcs[i] for i in liga_common_atoms]
ligs_common_atoms = [liga_common_atoms, ligb_common_atoms]

graphs = [make_graph(liga_top), make_graph(ligb_top)]


ligb_coor = align_coordinates(
    liga_coor, ligb_coor, liga_common_atoms, ligb_common_atoms
)

ligs_coor = [liga_coor, ligb_coor]

lambdas_list = [
    [(1.0, 1.0), (0.0, 0.0)],
    [(0.5, 1.0), (0.0, 0.0)],
    [(0.0, 1.0), (0.0, 0.0)],
    [(0.0, 0.8), (0.0, 0.2)],
    [(0.0, 0.6), (0.0, 0.4)],
    [(0.0, 0.4), (0.0, 0.6)],
    [(0.0, 0.2), (0.0, 0.8)],
    [(0.0, 0.0), (0.0, 1.0)],
    [(0.0, 0.0), (0.5, 1.0)],
    [(0.0, 0.0), (1.0, 1.0)],
]

for phase in ["vacuum", "water"]:
    os.makedirs(f"./output/{phase}_phase", exist_ok=True)
    with open(f"./output/{phase}_phase/lambdas.pkl", "wb") as f:
        pickle.dump(lambdas_list, f)

    for lambdas in lambdas_list:
        if phase == "vacuum":
            system_xml, top, coor = make_alchemical_system(
                ligs,
                [liga_top, ligb_top],
                ligs_common_atoms,
                ligs_coor,
                lambdas,
                None,
                None,
                None,
            )
        else:
            system_xml, top, coor = make_alchemical_system(
                ligs,
                [liga_top, ligb_top],
                ligs_common_atoms,
                ligs_coor,
                lambdas,
                envi,
                envi_top,
                envi_coor,
            )

        tree = ET.ElementTree(system_xml)
        ET.indent(tree.getroot())
        (elec0, vdw0), (elec1, vdw1) = lambdas
        os.makedirs(f"./output/{phase}_phase/sys", exist_ok=True)
        tree.write(
            f"./output/{phase}_phase/sys/{elec0:.2f}_{vdw0:.2f}_{elec1:.2f}_{vdw1:.2f}.xml",
            xml_declaration=True,
            method="xml",
            encoding="utf-8",
        )

        app.PDBFile.writeFile(
            top, coor * 10, f"./output/{phase}_phase/system.pdb", keepIds=True
        )

        with open(f"./output/{phase}_phase/topology.pkl", "wb") as file_handle:
            pickle.dump(top, file_handle)

        make_psf_from_topology(top, f"./output/{phase}_phase/topology.psf")

<class 'networkx.utils.decorators.argmap'> compilation 17:3: FutureWarning: 

single_target_shortest_path_length will return a dict instead of
an iterator in version 3.5


In [4]:
for phase, lambdas in product(["water", "vacuum"], lambdas_list):
    (elec0, vdw0), (elec1, vdw1) = lambdas
    lambdas_str = f"{elec0:.2f}_{vdw0:.2f}_{elec1:.2f}_{vdw1:.2f}"
    print(f"Running simulation for lambdas {lambdas_str} in {phase}", flush=True)

    ## deserialize the system
    with open(
        f"./output/{phase}_phase/sys/{elec0:.2f}_{vdw0:.2f}_{elec1:.2f}_{vdw1:.2f}.xml",
        "r",
    ) as f:
        system = XmlSerializer.deserialize(f.read())

    ## add barostat
    if phase == "water":
        system.addForce(mm.MonteCarloBarostat(1 * unit.atmospheres, 298.15 * unit.kelvin))

    with open(f"./output/{phase}_phase/topology.pkl", "rb") as f:
        topology = pickle.load(f)

    pdb = app.PDBFile(f"./output/{phase}_phase/system.pdb")

    integrator = mm.LangevinMiddleIntegrator(
        298.15 * unit.kelvin, 1.0 / unit.picosecond, 0.002 * unit.picoseconds
    )
    platform = mm.Platform.getPlatformByName("CUDA")
    simulation = app.Simulation(topology, system, integrator, platform)
    simulation.context.setPositions(pdb.positions)

    print("Minimizing energy ...", flush=True)
    simulation.minimizeEnergy()

    print("Equilibrating ...", flush=True)
    simulation.integrator.setStepSize(0.002 * unit.picoseconds)
    for T in np.linspace(50, 298.15, 5):
        integrator.setTemperature(T * unit.kelvin)
        simulation.context.setVelocitiesToTemperature(T * unit.kelvin)
        simulation.step(1_000)
    simulation.integrator.setStepSize(0.002 * unit.picoseconds)

    os.makedirs(f"./output/{phase}_phase/traj", exist_ok=True)
    simulation.reporters.append(
        app.DCDReporter(f"./output/{phase}_phase/traj/{lambdas_str}.dcd", 500)
    )

    print("Production run ...", flush=True)
    start_time = time.time()
    simulation.step(100_000)

    simulation.saveCheckpoint(f"./output/{phase}_phase/traj/{lambdas_str}.chk")
    print(f"Simulation finished in {time.time() - start_time:.2f} seconds")

Running simulation for lambdas 1.00_1.00_0.00_0.00 in water
Minimizing energy ...
Equilibrating ...
Production run ...
Simulation finished in 21.25 seconds
Running simulation for lambdas 0.50_1.00_0.00_0.00 in water
Minimizing energy ...
Equilibrating ...
Production run ...
Simulation finished in 21.23 seconds
Running simulation for lambdas 0.00_1.00_0.00_0.00 in water
Minimizing energy ...


KeyboardInterrupt: 

In [5]:
def compute_energy(phase, lambdas):
    (elec0, vdw0), (elec1, vdw1) = lambdas

    ## deserialize the system
    with open(
        f"./output/{phase}_phase/sys/{elec0:.2f}_{vdw0:.2f}_{elec1:.2f}_{vdw1:.2f}.xml",
        "r",
    ) as f:
        system = mm.XmlSerializer.deserialize(f.read())

    ## add barostat
    if phase == "water":
        system.addForce(mm.MonteCarloBarostat(1 * unit.atmospheres, 298.15 * unit.kelvin))

    with open(f"./output/{phase}_phase/topology.pkl", "rb") as f:
        topology = pickle.load(f)
    topology = mdtraj.Topology.from_openmm(topology)

    integrator = mm.LangevinMiddleIntegrator(
        298.15 * unit.kelvin, 1.0 / unit.picosecond, 0.002 * unit.picoseconds
    )
    kbT = 298.15 * unit.kelvin * unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA

    platform = mm.Platform.getPlatformByName("CUDA")
    simulation = app.Simulation(topology, system, integrator, platform)

    ## load trajectories
    reduced_u = []
    for lambdas in lambdas_list:
        (elec0, vdw0), (elec1, vdw1) = lambdas
        lambdas_str_traj = f"{elec0:.2f}_{vdw0:.2f}_{elec1:.2f}_{vdw1:.2f}"
        traj = mdtraj.load(
            f"./output/{phase}_phase/traj/{lambdas_str_traj}.dcd",
            top=topology,
        )
        
        for xyz, unit_cell_vectors in zip(traj.xyz, traj.unitcell_vectors):
            simulation.context.setPositions(xyz)
            simulation.context.setPeriodicBoxVectors(*unit_cell_vectors)
            u = simulation.context.getState(getEnergy=True).getPotentialEnergy() / kbT
            reduced_u.append(u)


    reduced_u = np.array(reduced_u)
    return reduced_u

In [6]:
reduced_u = {'water': [], 'vacuum': []}
for phase, lambdas in product(["water", "vacuum"], lambdas_list):
    u = compute_energy(phase, lambdas)
    reduced_u[phase].append(u)

In [7]:
Fs = {}
for phase in ["water", "vacuum"]:
    reduced_u[phase] = np.array(reduced_u[phase])
    u = reduced_u[phase]
    num_confs = np.array([u.shape[1]//u.shape[0] ] * u.shape[0])

    fastmbar = FastMBAR(u, num_confs, cuda=True, verbose=True, method="L-BFGS-B")

    kbT = 298.15 * unit.kelvin * unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
    kbT = kbT.value_in_unit(unit.kilocalorie_per_mole)
    Fs[phase] = fastmbar.F * kbT

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            9     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.53565D+04    |proj g|=  3.95161D-02

At iterate    1    f=  2.53565D+04    |proj g|=  2.34813D-02

At iterate    2    f=  2.53565D+04    |proj g|=  1.22244D-02

At iterate    3    f=  2.53565D+04    |proj g|=  1.48129D-02

At iterate    4    f=  2.53565D+04    |proj g|=  3.65514D-03

At iterate    5    f=  2.53565D+04    |proj g|=  1.90111D-03

At iterate    6    f=  2.53565D+04    |proj g|=  2.94193D-03

At iterate    7    f=  2.53565D+04    |proj g|=  1.73511D-03

At iterate    8    f=  2.53565D+04    |proj g|=  3.38467D-04

At iterate    9    f=  2.53565D+04    |proj g|=  2.63754D-05

At iterate   10    f=  2.53565D+04    |proj g|=  4.38010D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cau

 This problem is unconstrained.
 This problem is unconstrained.


In [8]:
F_water = Fs["water"]
F_vacuum = Fs["vacuum"]

dF_water = F_water[-1] - F_water[0]
dF_vacuum = F_vacuum[-1] - F_vacuum[0]

dF = dF_water - dF_vacuum
print(f"Free energy difference: {dF:.2f} kcal/mol")

Free energy difference: -4.70 kcal/mol
